In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
#from collections import defaultdict
import nltk
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

from bs4 import BeautifulSoup
from pandarallel import pandarallel

In [ ]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [ ]:
pandarallel.initialize(progress_bar=True,
                        nb_workers=6,
                        #verbose=1
                       )

In [ ]:
df = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/df_process_text.csv')

In [ ]:
df.head(30)

In [ ]:
df_least_freq_words = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/least_freq_words_to_drop.csv')

In [ ]:
df_most_freq_words = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/too_freq_words.csv')

In [ ]:
df_least_freq_words.head(10)

In [ ]:
df_most_freq_words.head(10)

In [ ]:
least_freq_words = list(df_least_freq_words['words'].values)
least_freq_words[:20]

In [ ]:
most_freq_words = list(df_most_freq_words['words'].values)
most_freq_words[:20]

In [ ]:
def process_text_2(doc,
                   rejoin=False,
                   list_rare_words=None,
                    min_len_word=2,
                    force_is_alpha=False,
                    eng_words = False
                 ) :
    """function of text processing

    positional arguments:
    ----------------------
    doc: str : the document (aka a text in str format) to process

    opt arguments:
    ---------------------
    rejoin: bool : if True return a string else return a list of tokens
    list_rare_words: list : a list of rare words to exclude
    min_len_words: int : the minimum lenght of words to not exclude
    force_is_alpha: int : if 1, exclude all tokens with a numeric character

    return:
    --------------------
    a string (if rejoin is True) or a list of tokens
    """

    #tokenizer
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)
    #
    # stop words:
    english_stop_words=nltk.corpus.stopwords.words('English')
    cleaned_tokens_list = [word for word in raw_tokens_list if word not in english_stop_words]

    #############################################################################################

    # no rare tokens
    # non_rare_tokens = [w for w in cleaned_tokens_list if w not in list_rare_words]

    # no more len words
    more_than_N = [w for w in cleaned_tokens_list if len(w) >= min_len_word]

    # only alpha characters
    if force_is_alpha:
        alpha_tokens = [w for w in more_than_N if w.isalpha()]
    else:
        alpha_tokens = more_than_N

    #############################################################################################

    #lemmatizer
    trans = WordNetLemmatizer()
    trans_text = [trans.lemmatize(word) for word in alpha_tokens]


    #############################################################################################

    good_words = remove_un(trans_text)


    least_freq = [w for w in raw_tokens_list if w not in least_freq_words]

    most_freq = [w for w in least_freq if w not in most_freq_words]

    if rejoin:
        return " ".join(most_freq)

    return most_freq

In [ ]:
def final_clean(doc):
    new_doc = process_text_2(doc,
                           rejoin=False,
                           )
    return new_doc

In [ ]:
df['_clean_text_2']= df['_clean_text_rejoin'].parallel_apply(lambda x : final_clean(x))

In [ ]:
len(df['_clean_text'][1])

In [ ]:
len(set(df['_clean_text_2'][1]))

In [ ]:
df['_clean_text_2'] = df['_clean_text_2'].apply(lambda x: set(x))
list(df['_clean_text_2'][0])

In [ ]:
df['_clean_text_2'] = df['_clean_text_2'].apply(lambda x: sorted(x))
df['_clean_text_2'][0]

In [ ]:
element = df['_clean_text_2'][0]

In [ ]:
list_1 = sorted(element)

In [ ]:
list_1

In [ ]:
len(df['_clean_text_2'][0])

In [ ]:
tmp = df._clean_text_2.str.len()

In [ ]:
tmp[:30]

In [ ]:
df["_len_txt_clean_2"] = tmp

In [ ]:
df.head(10)

In [ ]:
df.to_csv("data/cleaned/df_process_text_2.csv", index=False)

In [ ]:
list = list(df._clean_text_2.values)
list

In [ ]:
type(df['_clean_text_2'][0])

In [ ]:
corpus = [" ".join(row) for row in df['_clean_text_2']]
corpus[:30]

In [ ]:
df_text_rejoin_2 = pd.DataFrame({"_clean_text_rejoin_2" : corpus})
df_text_rejoin_2[:10]

In [ ]:
df = df.join(df_text_rejoin_2)
df.head(5)

In [ ]:
len(df['_clean_text'][0])

In [ ]:
len(df['_clean_text_2'][0])

In [ ]:
len(df['_clean_text'][0])

In [ ]:
len(set(df['_clean_text_2'][0]))

In [ ]:
df['_clean_text'][0]

In [ ]:
df['_clean_text_2'][0]

In [ ]:
len(df['_clean_text_rejoin'][0])

In [ ]:
len(df['_clean_text_rejoin_2'][0])

In [ ]:
corpus_final = ''.join(corpus)
corpus_final[:1000]

In [ ]:
len(set(corpus_final))

In [ ]:
len(corpus_final)

In [ ]:
tmp_text_2 = pd.Series(corpus_final).value_counts()
tmp_text_2[:30]

In [ ]:
tmp_text_2[:30]

In [ ]:
df.to_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/df_process_text_2.csv', index=False)

In [ ]:
df['_clean_text_rejoin_2']= df['_clean_text_rejoin'].parallel_apply(lambda x : final_clean(x))